In [47]:
# debug_and_fix_tags.py
import zmq, msgpack, msgpack_numpy as m
m.patch()
import numpy as np, time, math

PORT = 5557
TAG4_ID = 4
TAG4_HEIGHT_M = 0.075  # same as your system
UP = np.array([0.0, 0.0, 1.0])

ctx = zmq.Context()
sock = ctx.socket(zmq.SUB)
sock.setsockopt(zmq.SUBSCRIBE, b"")
sock.connect(f"tcp://127.0.0.1:{PORT}")

def rot_err(R):
    I = np.eye(3)
    return np.linalg.norm(R.dot(R.T) - I), np.linalg.det(R)

def vec_info(name, v):
    return f"{name}: [{v[0]:+.3f}, {v[1]:+.3f}, {v[2]:+.3f}] | |v|={np.linalg.norm(v):.3f}"

def align_local_z_to_world_up(T):
    """
    Returns T_fixed = T.dot(R_local) where R_local rotates tag-local axes
    so that tag-local +Z (0,0,1) maps to world UP.
    This preserves tag position but changes its orientation so its Z aligns to UP.
    """
    R = T[:3,:3]
    n = R[:,2]  # current tag Z in world
    n = n / np.linalg.norm(n)
    # target k in tag-local coordinates: we want R * (R_local * e_z) = UP
    # So R_local * e_z = R.T @ UP = k_local
    k_local = R.T.dot(UP)
    k_local = k_local / np.linalg.norm(k_local)
    # now compute rotation that maps e_z -> k_local in local coordinates
    ez = np.array([0.0, 0.0, 1.0])
    v = np.cross(ez, k_local)
    s = np.linalg.norm(v)
    c = np.dot(ez, k_local)
    if s < 1e-8:
        # already aligned or opposite
        if c > 0:
            R_local = np.eye(3)
        else:
            # 180 deg rotation about any orth axis (choose x)
            R_local = np.array([[1,0,0],[0,-1,0],[0,0,-1]], float)
    else:
        vx = np.array([[0, -v[2], v[1]],[v[2], 0, -v[0]],[-v[1], v[0], 0]], float)
        R_local = np.eye(3) + vx + vx.dot(vx) * ((1 - c) / (s**2))
    T_fixed = np.eye(4)
    T_fixed[:3,:3] = R.dot(R_local)  # new world rotation
    T_fixed[:3,3] = T[:3,3].copy()
    return T_fixed

print("[debug] waiting for snapshot...")
msg = sock.recv()   # blocking for one snapshot
data = msgpack.unpackb(msg, object_hook=m.decode, strict_map_key=False)

tags = data.get("tags", {})
cams = data.get("cameras", {})

print("--- SNAPSHOT CHECK ---\n")
for tid, T in tags.items():
    T = np.array(T, dtype=float)
    pos = T[:3,3]
    R = T[:3,:3]
    err, det = rot_err(R)
    n = R[:,2]  # tag local +Z in world
    print(f"TAG {tid}")
    print(f"  pos = {pos}")
    print(f"  normal (world) = {n}  | dot(up) = {n.dot(UP):.3f}")
    print(f"  rot ortho err = {err:.6f}, det = {det:.6f}")
    # check if tag is floor-like (z small) or mobile
    z = pos[2]
    print(f"  z = {z:.4f} m")
    # decide if orientation is bad for a floor tag
    if tid != TAG4_ID:
        if abs(z) < 0.5:
            # expected floor tag -> normal dot up should be near ±1
            if abs(n.dot(UP)) < 0.8:
                print("  >>> WARNING: tag normal not aligned with UP (likely tilt/vertical).")
                T_fix = align_local_z_to_world_up(T)
                nfix = T_fix[:3,:3][:,2]
                print("    -> Suggest fixed normal (world) = ", nfix, "pos unchanged:", T_fix[:3,3])
            else:
                print("  orientation looks OK for floor tag.")
        else:
            print("  tag has high z; may be on raised surface (bed/shelf).")
    else:
        # mobile tag: check height near TAG4_HEIGHT_M
        if abs(z - TAG4_HEIGHT_M) > 0.05:
            print(f"  >>> WARNING: mobile tag z != {TAG4_HEIGHT_M:.3f} m (delta {z-TAG4_HEIGHT_M:+.3f})")
            # give fix that sets z to expected height but keeps orientation
            T_fixed = T.copy()
            T_fixed[2,3] = TAG4_HEIGHT_M
            print("    -> suggested T with corrected height (translation):", T_fixed[:3,3])
        # mobile orientation check (often vertical or horizontal depending on mount)
        print("  mobile tag normal dot(up) = ", n.dot(UP))

print("\nCameras:")
for cname, T in cams.items():
    T = np.array(T, dtype=float)
    pos = T[:3,3]
    R = T[:3,:3]
    err, det = rot_err(R)
    print(f"CAM {cname}: pos={pos}, rot_err={err:.6f}, det={det:.6f}")

print("\n--- DONE ---")


[debug] waiting for snapshot...
--- SNAPSHOT CHECK ---

TAG 2
  pos = [0.93746742 0.8630616  0.07352976]
  normal (world) = [-0.10142923  0.01906146  0.99466013]  | dot(up) = 0.995
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0735 m
  orientation looks OK for floor tag.
TAG 0
  pos = [0.92425455 0.06736595 0.18822022]
  normal (world) = [-0.04370148 -0.03768133  0.99833376]  | dot(up) = 0.998
  rot ortho err = 0.000000, det = 1.000000
  z = 0.1882 m
  orientation looks OK for floor tag.
TAG 3
  pos = [ 0.09503389  0.8991233  -0.02109775]
  normal (world) = [-0.16328576  0.0226556   0.98631865]  | dot(up) = 0.986
  rot ortho err = 0.000000, det = 1.000000
  z = -0.0211 m
  orientation looks OK for floor tag.
TAG 1
  pos = [0. 0. 0.]
  normal (world) = [-4.12619948e-18  2.55483077e-16  1.00000000e+00]  | dot(up) = 1.000
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0000 m
  orientation looks OK for floor tag.
TAG 5
  pos = [0.74497135 0.56011145 0.05974904]
  normal (world) = 

In [1]:
import open3d as o3d
import numpy as np
import pandas as pd
import time
import sys
import os
import glob

# ---------- Configuration ----------
BALL_RADIUS = 0.025
CAM1_POS = [0.15, -1.4, 2.0]
CAM2_POS = [1.1, 1.8, 2.0]
LOOP_DELAY = 5.0
MIN_MOVE_DIST = 0.0001 

# ---------- Helper Functions ----------
def create_custom_grid(size=4.0, step=0.3):
    lines = []
    points = []
    start = -size
    end = size
    num_steps = int((end - start) / step) + 1
    
    for i in range(num_steps):
        y = start + i * step
        points.append([-size, y, 0]); points.append([size, y, 0])
        lines.append([len(points)-2, len(points)-1])
        
        x = start + i * step
        points.append([x, -size, 0]); points.append([x, size, 0])
        lines.append([len(points)-2, len(points)-1])

    colors = [[0.3, 0.3, 0.3] for _ in range(len(lines))]
    grid = o3d.geometry.LineSet()
    grid.points = o3d.utility.Vector3dVector(points)
    grid.lines = o3d.utility.Vector2iVector(lines)
    grid.colors = o3d.utility.Vector3dVector(colors)
    return grid

def create_robot_mesh():
    robot_box = o3d.geometry.TriangleMesh.create_box(0.25, 0.25, 0.075)
    robot_box.translate([-0.2, -0.15, 0])
    robot_box.paint_uniform_color([0.1, 0.1, 0.9]) 
    dustbin = o3d.geometry.TriangleMesh.create_cylinder(radius=0.06, height=0.12)
    dustbin.translate([-0.05, 0, 0.075])
    dustbin.paint_uniform_color([0.9, 0.2, 0.2]) 
    robot_mesh = robot_box + dustbin
    robot_mesh.compute_vertex_normals()
    return robot_mesh

def compute_robot_center(row):
    # Extract Tag 4
    p4 = None
    if not pd.isna(row['tag4_x']) and str(row['tag4_x']) != '':
        p4 = np.array([float(row['tag4_x']), float(row['tag4_y']), float(row['tag4_z'])])

    # Extract Tag 5
    p5 = None
    if not pd.isna(row['tag5_x']) and str(row['tag5_x']) != '':
        p5 = np.array([float(row['tag5_x']), float(row['tag5_y']), float(row['tag5_z'])])

    shift_m = 0.096
    default_y_axis = np.array([0.0, 1.0, 0.0])

    if p4 is not None and p5 is not None:
        return 0.5 * (p4 + p5)
    if p4 is not None:
        return p4 + (-shift_m) * default_y_axis
    if p5 is not None:
        return p5 + (shift_m) * default_y_axis
    return None

# ---------- Main App Logic ----------
class LogVisualizerLegacy:
    def __init__(self):
        # Data Loading
        self.data = self.find_and_load_csv()
        self.frame_idx = 0
        self.fps = 30
        self.is_paused = False
        self.ball_path_points = []
        
        # Geometry Placeholders
        self.ball_geom = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS)
        self.ball_geom.paint_uniform_color([1.0, 0.5, 0.0])
        self.ball_geom.compute_vertex_normals()
        
        self.robot_geom = create_robot_mesh()
        
        # CHANGED: Use PointCloud instead of LineSet
        self.trail_geom = o3d.geometry.PointCloud()

        # Initialize Visualizer
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window(window_name="Ball Catcher Replay (Legacy)", width=1280, height=720)

        # Increase point size so the trail is visible
        opt = self.vis.get_render_option()
        opt.point_size = 5.0
        opt.background_color = np.array([0.1, 0.1, 0.1])

        # Add Geometries
        self.vis.add_geometry(self.ball_geom)
        self.vis.add_geometry(self.robot_geom)
        self.vis.add_geometry(create_custom_grid())
        self.vis.add_geometry(o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5))
        
        # Add Cameras
        for i, pos in enumerate([CAM1_POS, CAM2_POS]):
            cam = o3d.geometry.TriangleMesh.create_cone(radius=0.1, height=0.2)
            cam.paint_uniform_color([0,1,0] if i==1 else [1,0,0])
            cam.compute_vertex_normals()
            t = np.eye(4); t[:3,3] = pos
            # Rotate cone to point down
            R = cam.get_rotation_matrix_from_xyz((np.pi,0,0))
            cam.rotate(R, center=(0,0,0))
            cam.transform(t)
            self.vis.add_geometry(cam)

        # View Control
        ctr = self.vis.get_view_control()
        ctr.set_lookat([0, 0, 0])
        ctr.set_front([0, -1, 1]) # Look from y=-1, z=1 direction
        ctr.set_up([0, 0, 1])
        ctr.set_zoom(0.8)

        # Register Callbacks
        self.vis.register_key_callback(ord(' '), self.toggle_pause)
        self.vis.register_key_callback(ord('+'), self.increase_fps)
        self.vis.register_key_callback(ord('='), self.increase_fps) # For keyboards where + is =
        self.vis.register_key_callback(ord('-'), self.decrease_fps)
        self.vis.register_animation_callback(self.update_frame)

    def find_and_load_csv(self):
        files = glob.glob("../../data/triangulation_logs/log_1763931870.csv")
        if not files: print("No CSV found!"); sys.exit()
        latest = max(files, key=os.path.getmtime)
        print(f"Loading: {latest}")
        return pd.read_csv(latest)

    def toggle_pause(self, vis):
        self.is_paused = not self.is_paused
        print(f"Paused: {self.is_paused}")
        return False

    def increase_fps(self, vis):
        self.fps = min(65, self.fps + 5)
        print(f"FPS: {self.fps}")
        return False

    def decrease_fps(self, vis):
        self.fps = max(5, self.fps - 5)
        print(f"FPS: {self.fps}")
        return False

    def update_frame(self, vis):
        if self.is_paused: return False
        
        # Simple Frame Throttling
        time.sleep(1.0 / self.fps)

        if self.frame_idx >= len(self.data):
            print(f"Looping in {LOOP_DELAY}s...")
            time.sleep(LOOP_DELAY)
            self.frame_idx = 0
            self.ball_path_points = []
            
            # Clear trail visual (Empty PointCloud)
            self.trail_geom.points = o3d.utility.Vector3dVector([])
            vis.update_geometry(self.trail_geom)
            return True

        row = self.data.iloc[self.frame_idx]
        
        # 1. Update Ball
        bx, by, bz = row['ball_3d_x'], row['ball_3d_y'], row['ball_3d_z']
        if not pd.isna(bx) and str(bx) != '':
            pos = np.array([float(bx), float(by), float(bz)])
            # Translate relative to current center (reset to 0 then move)
            center = self.ball_geom.get_center()
            self.ball_geom.translate(pos - center)
            vis.update_geometry(self.ball_geom)
            
            # Update Trail
            self.update_trail(vis, pos)
        else:
            # Move away
            center = self.ball_geom.get_center()
            self.ball_geom.translate(np.array([0,0,-100]) - center)
            vis.update_geometry(self.ball_geom)

        # 2. Update Robot
        rpos = compute_robot_center(row)
        if rpos is not None:
            center = self.robot_geom.get_center()
            self.robot_geom.translate(rpos - center)
            vis.update_geometry(self.robot_geom)
        
        self.frame_idx += 1
        return True

    def update_trail(self, vis, new_pt):
        # Add point logic
        if len(self.ball_path_points) > 0:
            dist = np.linalg.norm(new_pt - self.ball_path_points[-1])
            if dist < MIN_MOVE_DIST: return

        self.ball_path_points.append(new_pt)
        
        # Update PointCloud Geometry
        points = np.array(self.ball_path_points)
        self.trail_geom.points = o3d.utility.Vector3dVector(points)
        self.trail_geom.paint_uniform_color([0, 1, 0]) # Green
        
        # If it's the first time adding points, we might need to add geometry
        if len(self.ball_path_points) == 1:
             vis.add_geometry(self.trail_geom, reset_bounding_box=False)
        else:
             vis.update_geometry(self.trail_geom)

    def run(self):
        self.vis.run()
        self.vis.destroy_window()

if __name__ == "__main__":
    app = LogVisualizerLegacy()
    app.run()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading: ../../data/triangulation_logs/log_1763931870.csv
Looping in 5.0s...
Looping in 5.0s...


In [1]:
import open3d as o3d
import numpy as np
import pandas as pd
import time
import sys
import os
import glob

# ---------- Configuration ----------
BALL_RADIUS = 0.025
RIM_HEIGHT = 0.20  # 20cm Target Height (Red Grid)
GRAVITY = 9.81

# Colors
COLOR_RAW = [1.0, 0.5, 0.0]   # Orange (Actual Detection)
COLOR_LKF = [0.1, 0.1, 0.9]   # Blue (Linear Kalman)
COLOR_EKF = [0.8, 0.0, 0.8]   # Purple (Extended Kalman)
COLOR_RLS = [0.0, 0.8, 0.0]   # Green (Recursive Least Squares)

# Hidden dummy point to prevent Open3D "0 points" warning
DUMMY_POINT = np.array([[0.0, 0.0, -99.0]])

# ---------- Filters ----------

class LinearKalmanFilter:
    def __init__(self, dt=1/30.0):
        # State: [x, y, z, vx, vy, vz]
        self.x = np.zeros(6)
        self.P = np.eye(6) * 100.0
        self.Q = np.eye(6) * 0.01
        self.Q[3:, 3:] *= 0.1 
        self.R = np.eye(3) * 0.05
        self.H = np.zeros((3, 6))
        self.H[0,0] = 1; self.H[1,1] = 1; self.H[2,2] = 1
        self.last_ts = None

    def predict(self, ts):
        if self.last_ts is None: dt = 1/30.0
        else: dt = ts - self.last_ts
        self.last_ts = ts

        F = np.eye(6)
        F[0, 3] = dt; F[1, 4] = dt; F[2, 5] = dt
        
        self.x = F @ self.x
        self.x[5] -= GRAVITY * dt
        self.P = F @ self.P @ F.T + self.Q

    def update(self, meas):
        z = np.array(meas)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(6) - K @ self.H) @ self.P
        return self.x[:3]

class ExtendedKalmanFilter:
    def __init__(self):
        self.x = np.zeros(6) 
        self.P = np.eye(6) * 100.0
        self.Q = np.eye(6) * 0.01
        self.R = np.eye(3) * 0.05
        self.H = np.zeros((3, 6))
        self.H[0,0] = 1; self.H[1,1] = 1; self.H[2,2] = 1
        self.last_ts = None
        self.drag_coeff = 0.05 

    def predict(self, ts):
        if self.last_ts is None: dt = 1/30.0
        else: dt = ts - self.last_ts
        self.last_ts = ts
        
        x, y, z, vx, vy, vz = self.x
        new_vx = vx + (-self.drag_coeff * vx) * dt
        new_vy = vy + (-self.drag_coeff * vy) * dt
        new_vz = vz + (-GRAVITY - self.drag_coeff * vz) * dt
        new_x = x + vx * dt
        new_y = y + vy * dt
        new_z = z + vz * dt
        
        self.x = np.array([new_x, new_y, new_z, new_vx, new_vy, new_vz])

        F = np.eye(6)
        F[0,3] = dt; F[1,4] = dt; F[2,5] = dt
        F[3,3] = 1 - self.drag_coeff * dt
        F[4,4] = 1 - self.drag_coeff * dt
        F[5,5] = 1 - self.drag_coeff * dt
        
        self.P = F @ self.P @ F.T + self.Q

    def update(self, meas):
        z = np.array(meas)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(6) - K @ self.H) @ self.P
        return self.x[:3]

class RecursiveLeastSquares:
    def __init__(self):
        self.theta_x = np.zeros((2,1))
        self.P_x = np.eye(2) * 1000
        self.theta_y = np.zeros((2,1))
        self.P_y = np.eye(2) * 1000
        self.theta_z = np.zeros((3,1)) 
        self.P_z = np.eye(3) * 1000
        
        # TUNING: 0.85 for responsive tracking
        self.lambda_factor = 0.85 
        self.start_time = None
        self.path = []
        self.current_state = np.zeros(6)

    def update(self, ts, meas):
        if self.start_time is None: self.start_time = ts
        t = ts - self.start_time
        x, y, z = meas

        # Linear Fit for X, Y
        phi_lin = np.array([[1], [t]])
        self.theta_x, self.P_x = self._rls_step(self.theta_x, self.P_x, phi_lin, x)
        self.theta_y, self.P_y = self._rls_step(self.theta_y, self.P_y, phi_lin, y)
        
        # Quadratic Fit for Z
        phi_quad = np.array([[1], [t], [t**2]])
        self.theta_z, self.P_z = self._rls_step(self.theta_z, self.P_z, phi_quad, z)
        
        # Lookahead compensation
        lag_comp = 0.02
        state = self.get_state(ts + lag_comp)
        
        self.current_state = state
        self.path.append(state[:3].copy())
        return state

    def _rls_step(self, theta, P, phi, val):
        num = P @ phi
        den = self.lambda_factor + phi.T @ P @ phi
        K = num / den
        err = val - phi.T @ theta
        theta_new = theta + K * err
        P_new = (P - K @ phi.T @ P) / self.lambda_factor
        return theta_new, P_new

    def get_state(self, ts):
        if self.start_time is None: return np.zeros(6)
        t = ts - self.start_time
        
        px = (np.array([[1, t]]) @ self.theta_x)[0,0]
        py = (np.array([[1, t]]) @ self.theta_y)[0,0]
        pz = (np.array([[1, t, t**2]]) @ self.theta_z)[0,0]
        
        vx = self.theta_x[1,0]
        vy = self.theta_y[1,0]
        vz = self.theta_z[1,0] + 2*self.theta_z[2,0]*t
        
        return np.array([px, py, pz, vx, vy, vz])
    
    # Helper to push an interpolated state to path
    def interpolate_to_path(self, ts):
        # We query the polynomial at the interpolated time
        lag_comp = 0.02
        s = self.get_state(ts + lag_comp)
        self.path.append(s[:3].copy())
        return s[:3]

# ---------- Helper Functions ----------
def create_grid():
    grid = o3d.geometry.LineSet()
    points = []
    lines = []
    step = 0.5
    size = 3.0
    
    # 1. Floor Grid (Grey) - Z=0
    for i in range(int(-size/step), int(size/step)+1):
        x = i * step
        points.append([x, -size, 0]); points.append([x, size, 0])
        lines.append([len(points)-2, len(points)-1])
        points.append([-size, x, 0]); points.append([size, x, 0])
        lines.append([len(points)-2, len(points)-1])
    
    # 2. Target Height Plane (Red-ish) - Z=RIM_HEIGHT
    # This helps you visualize where the ball lands relative to the rim
    offset = len(points)
    for i in range(int(-size/step), int(size/step)+1):
        x = i * step
        points.append([x, -size, RIM_HEIGHT]); points.append([x, size, RIM_HEIGHT])
        lines.append([len(points)-2, len(points)-1])
        points.append([-size, x, RIM_HEIGHT]); points.append([size, x, RIM_HEIGHT])
        lines.append([len(points)-2, len(points)-1])

    colors = [[0.3, 0.3, 0.3] for _ in range(offset)] + [[0.5, 0.1, 0.1] for _ in range(len(lines)-offset)]
    
    grid.points = o3d.utility.Vector3dVector(points)
    grid.lines = o3d.utility.Vector2iVector(lines)
    grid.colors = o3d.utility.Vector3dVector(colors)
    return grid

def predict_landing(state_vec, dt_step=0.02):
    x, y, z, vx, vy, vz = state_vec
    traj_points = []
    
    # If already below rim and falling, no valid landing prediction needed
    if z < RIM_HEIGHT and vz < 0:
        return []

    curr_x, curr_y, curr_z = x, y, z
    curr_vx, curr_vy, curr_vz = vx, vy, vz
    
    sim_t = 0
    # Predict for up to 2 seconds into the future
    while curr_z > 0 and sim_t < 2.0: 
        traj_points.append([curr_x, curr_y, curr_z])
        
        # Stop exactly when we hit RIM_HEIGHT while falling
        if curr_z <= RIM_HEIGHT and curr_vz < 0:
            break
            
        curr_x += curr_vx * dt_step
        curr_y += curr_vy * dt_step
        curr_z += curr_vz * dt_step
        curr_vz -= GRAVITY * dt_step
        sim_t += dt_step
        
    return traj_points

# ---------- Main Visualizer ----------

class DetectionVisualizer:
    def __init__(self):
        self.data = self.load_data()
        self.idx = 0
        self.paused = False
        
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window("Filter Comparison", 1280, 720)
        
        self.ball = o3d.geometry.TriangleMesh.create_sphere(BALL_RADIUS)
        self.ball.paint_uniform_color(COLOR_RAW)
        self.ball.compute_vertex_normals()
        
        # Initialize Trails
        self.trails = {
            'raw': o3d.geometry.PointCloud(),
            'lkf': o3d.geometry.PointCloud(),
            'ekf': o3d.geometry.PointCloud(),
            'rls': o3d.geometry.PointCloud(),
            'pred_lkf': o3d.geometry.LineSet(),
            'pred_ekf': o3d.geometry.LineSet(),
            'pred_rls': o3d.geometry.LineSet()
        }
        
        # Prevent "0 points" warning by initializing with dummy hidden data
        for k in self.trails:
            self.trails[k].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            if isinstance(self.trails[k], o3d.geometry.LineSet):
                self.trails[k].lines = o3d.utility.Vector2iVector([])

        self.points_history = {'raw': [], 'lkf': [], 'ekf': [], 'rls': []}
        
        self.vis.add_geometry(self.ball)
        self.vis.add_geometry(create_grid())
        self.vis.add_geometry(o3d.geometry.TriangleMesh.create_coordinate_frame(0.3))
        
        for k in self.trails:
            self.vis.add_geometry(self.trails[k])

        self.lkf = LinearKalmanFilter()
        self.ekf = ExtendedKalmanFilter()
        self.rls = RecursiveLeastSquares()
        self.filters_initialized = False

        self.vis.register_key_callback(ord(' '), self.toggle_pause)
        self.vis.register_animation_callback(self.update)
        
        ctr = self.vis.get_view_control()
        ctr.set_lookat([0.5, 0, 0.5])
        ctr.set_front([-0.5, -1.0, 0.5])
        ctr.set_up([0, 0, 1])
        ctr.set_zoom(0.6)

    def load_data(self):
        # Use glob to find CSVs
        files = glob.glob("../../data/triangulation_logs/log_1764153186.csv")
        if not files: print("No CSV found"); sys.exit()
        latest = max(files, key=os.path.getmtime)
        print(f"Loading {latest}")
        
        # Load and clean data (coerce errors to NaN)
        df = pd.read_csv(latest)
        cols = ['ball_3d_x', 'ball_3d_y', 'ball_3d_z', 'timestamp']
        for c in cols:
            df[c] = pd.to_numeric(df[c], errors='coerce')
        return df

    def toggle_pause(self, vis):
        self.paused = not self.paused
        return False

    def update_trail_geom(self, name, points, color):
        # Fix for Open3D Warning: Never send empty array
        if not points or len(points) == 0:
            p_data = DUMMY_POINT
        else:
            p_data = np.array(points)
            
        self.trails[name].points = o3d.utility.Vector3dVector(p_data)
        self.trails[name].paint_uniform_color(color)
        self.vis.update_geometry(self.trails[name])

    def update_pred_line(self, name, points, color):
        # Fix for Open3D Warning: Need at least 2 points for a line
        if len(points) < 2:
            self.trails[name].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            self.trails[name].lines = o3d.utility.Vector2iVector([])
        else:
            lines = [[i, i+1] for i in range(len(points)-1)]
            self.trails[name].points = o3d.utility.Vector3dVector(np.array(points))
            self.trails[name].lines = o3d.utility.Vector2iVector(lines)
            self.trails[name].colors = o3d.utility.Vector3dVector([color for _ in lines])
            
        self.vis.update_geometry(self.trails[name])

    def update(self, vis):
        if self.paused: return False
        
        # Loop logic
        if self.idx >= len(self.data):
            time.sleep(1.0)
            self.idx = 0
            self.points_history = {k: [] for k in self.points_history}
            self.lkf = LinearKalmanFilter()
            self.ekf = ExtendedKalmanFilter()
            self.rls = RecursiveLeastSquares()
            self.filters_initialized = False
            # Reset trails to dummy
            for k in self.trails:
                self.trails[k].points = o3d.utility.Vector3dVector(DUMMY_POINT)
                if isinstance(self.trails[k], o3d.geometry.LineSet):
                    self.trails[k].lines = o3d.utility.Vector2iVector([])
                vis.update_geometry(self.trails[k])
            return True

        row = self.data.iloc[self.idx]
        
        # Check for valid data (NaNs were coerced in load_data)
        if pd.isna(row['ball_3d_x']):
            self.idx += 1
            return True
            
        bx, by, bz = row['ball_3d_x'], row['ball_3d_y'], row['ball_3d_z']
        ts = row['timestamp']
        meas = np.array([bx, by, bz])
        
        # Initialize filters on first valid point
        if not self.filters_initialized:
            self.lkf.x[:3] = meas; self.ekf.x[:3] = meas
            self.rls = RecursiveLeastSquares() 
            self.filters_initialized = True
            self.lkf.last_ts = ts; self.ekf.last_ts = ts

        # 1. Predict
        self.lkf.predict(ts)
        self.ekf.predict(ts)
        
        # 2. Update
        lkf_pos = self.lkf.update(meas)
        ekf_pos = self.ekf.update(meas)
        rls_state = self.rls.update(ts, meas)
        rls_pos = rls_state[:3]

        # 3. Store History for Tracing
        self.points_history['raw'].append(meas)
        self.points_history['lkf'].append(lkf_pos)
        self.points_history['ekf'].append(ekf_pos)
        self.points_history['rls'].append(rls_pos)
        
        # 4. Move Ball
        center = self.ball.get_center()
        self.ball.translate(meas - center)
        vis.update_geometry(self.ball)

        # 5. Future Predictions
        lkf_traj = predict_landing(self.lkf.x)
        ekf_traj = predict_landing(self.ekf.x)
        rls_traj = predict_landing(rls_state)

        self.update_pred_line('pred_lkf', lkf_traj, COLOR_LKF)
        self.update_pred_line('pred_ekf', ekf_traj, COLOR_EKF)
        self.update_pred_line('pred_rls', rls_traj, COLOR_RLS)

        # 6. Draw Trails
        self.update_trail_geom('raw', self.points_history['raw'], COLOR_RAW)
        self.update_trail_geom('lkf', self.points_history['lkf'], COLOR_LKF)
        self.update_trail_geom('ekf', self.points_history['ekf'], COLOR_EKF)
        self.update_trail_geom('rls', self.points_history['rls'], COLOR_RLS)

        self.idx += 1
        time.sleep(0.03) 
        return True

    def run(self):
        self.vis.run()
        self.vis.destroy_window()

if __name__ == "__main__":
    app = DetectionVisualizer()
    app.run()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loading ../../data/triangulation_logs/log_1764153186.csv
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [Sim

In [4]:
import open3d as o3d
import numpy as np
import pandas as pd
import time
import sys
import os
import glob

# ---------- Configuration ----------
BALL_RADIUS = 0.025
RIM_HEIGHT = 0.20  # 20cm Target Height
GRAVITY = 9.81
CAM1_POS = [0.15, -1.4, 2.0]
CAM2_POS = [1.1, 1.8, 2.0]
LOOP_DELAY = 5.0
MIN_MOVE_DIST = 0.0001 
DUMMY_POS = np.array([0.0, 0.0, -100.0])  # Hidden position far below

# Gap Filling Config
GAP_THRESHOLD = 0.04  # If gap > 40ms, start filling
INTERPOLATION_STEP = 0.01 # Generate a point every 10ms during gaps

# ---------- BRIGHT NEON COLORS ----------
COLOR_RAW = [1.0, 0.85, 0.0]   # Neon Gold/Yellow
COLOR_LKF = [0.0, 1.0, 1.0]    # Neon Cyan
COLOR_EKF = [1.0, 0.0, 1.0]    # Neon Magenta
COLOR_RLS = [0.0, 1.0, 0.0]    # Neon Lime Green
COLOR_PRED = [1.0, 0.2, 0.2]   # Bright Red for Landing

# ---------- Filter Logic ----------

class LinearKalmanFilter:
    def __init__(self, dt=1/30.0):
        self.x = np.zeros(6) # [x, y, z, vx, vy, vz]
        self.P = np.eye(6) * 100.0
        self.Q = np.eye(6) * 0.01
        self.Q[3:, 3:] *= 0.1 
        self.R = np.eye(3) * 0.05
        self.H = np.zeros((3, 6))
        self.H[0,0] = 1; self.H[1,1] = 1; self.H[2,2] = 1
        self.last_ts = None
        self.path = []

    def predict(self, ts):
        if self.last_ts is None: dt = 1/30.0
        else: dt = ts - self.last_ts
        
        # Prevent zero division or tiny steps if called redundantly
        if dt <= 1e-6: return

        self.last_ts = ts
        F = np.eye(6)
        F[0, 3] = dt; F[1, 4] = dt; F[2, 5] = dt
        self.x = F @ self.x
        self.x[5] -= GRAVITY * dt
        self.P = F @ self.P @ F.T + self.Q

    def update(self, meas):
        z = np.array(meas)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(6) - K @ self.H) @ self.P
        self.path.append(self.x[:3].copy())
        return self.x[:3]
    
    # Helper to just save current state to path (for interpolation)
    def save_state_to_path(self):
        self.path.append(self.x[:3].copy())

class ExtendedKalmanFilter:
    def __init__(self):
        self.x = np.zeros(6) 
        self.P = np.eye(6) * 100.0
        self.Q = np.eye(6) * 0.01
        self.R = np.eye(3) * 0.05
        self.H = np.zeros((3, 6))
        self.H[0,0] = 1; self.H[1,1] = 1; self.H[2,2] = 1
        self.last_ts = None
        self.drag_coeff = 0.05 
        self.path = []

    def predict(self, ts):
        if self.last_ts is None: dt = 1/30.0
        else: dt = ts - self.last_ts

        if dt <= 1e-6: return

        self.last_ts = ts
        x, y, z, vx, vy, vz = self.x
        new_vx = vx + (-self.drag_coeff * vx) * dt
        new_vy = vy + (-self.drag_coeff * vy) * dt
        new_vz = vz + (-GRAVITY - self.drag_coeff * vz) * dt
        new_x = x + vx * dt
        new_y = y + vy * dt
        new_z = z + vz * dt
        self.x = np.array([new_x, new_y, new_z, new_vx, new_vy, new_vz])
        F = np.eye(6)
        F[0,3] = dt; F[1,4] = dt; F[2,5] = dt
        F[3,3] = 1 - self.drag_coeff * dt
        F[4,4] = 1 - self.drag_coeff * dt
        F[5,5] = 1 - self.drag_coeff * dt
        self.P = F @ self.P @ F.T + self.Q

    def update(self, meas):
        z = np.array(meas)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(6) - K @ self.H) @ self.P
        self.path.append(self.x[:3].copy())
        return self.x[:3]
    
    def save_state_to_path(self):
        self.path.append(self.x[:3].copy())

class RecursiveLeastSquares:
    def __init__(self):
        self.theta_x = np.zeros((2,1))
        self.P_x = np.eye(2) * 1000
        self.theta_y = np.zeros((2,1))
        self.P_y = np.eye(2) * 1000
        self.theta_z = np.zeros((3,1)) 
        self.P_z = np.eye(3) * 1000
        
        # TUNING: 0.85 for responsive tracking
        self.lambda_factor = 0.85 
        self.start_time = None
        self.path = []
        self.current_state = np.zeros(6)

    def update(self, ts, meas):
        if self.start_time is None: self.start_time = ts
        t = ts - self.start_time
        x, y, z = meas

        # Linear Fit for X, Y
        phi_lin = np.array([[1], [t]])
        self.theta_x, self.P_x = self._rls_step(self.theta_x, self.P_x, phi_lin, x)
        self.theta_y, self.P_y = self._rls_step(self.theta_y, self.P_y, phi_lin, y)
        
        # Quadratic Fit for Z
        phi_quad = np.array([[1], [t], [t**2]])
        self.theta_z, self.P_z = self._rls_step(self.theta_z, self.P_z, phi_quad, z)
        
        # Lookahead compensation
        lag_comp = 0.02
        state = self.get_state(ts + lag_comp)
        
        self.current_state = state
        self.path.append(state[:3].copy())
        return state

    def _rls_step(self, theta, P, phi, val):
        num = P @ phi
        den = self.lambda_factor + phi.T @ P @ phi
        K = num / den
        err = val - phi.T @ theta
        theta_new = theta + K * err
        P_new = (P - K @ phi.T @ P) / self.lambda_factor
        return theta_new, P_new

    def get_state(self, ts):
        if self.start_time is None: return np.zeros(6)
        t = ts - self.start_time
        
        px = (np.array([[1, t]]) @ self.theta_x)[0,0]
        py = (np.array([[1, t]]) @ self.theta_y)[0,0]
        pz = (np.array([[1, t, t**2]]) @ self.theta_z)[0,0]
        
        vx = self.theta_x[1,0]
        vy = self.theta_y[1,0]
        vz = self.theta_z[1,0] + 2*self.theta_z[2,0]*t
        
        return np.array([px, py, pz, vx, vy, vz])
    
    # Helper to push an interpolated state to path
    def interpolate_to_path(self, ts):
        # We query the polynomial at the interpolated time
        lag_comp = 0.02
        s = self.get_state(ts + lag_comp)
        self.path.append(s[:3].copy())
        return s[:3]

# ---------- Helper Functions ----------
def create_custom_grid(size=4.0, step=0.3):
    lines = []
    points = []
    start = -size
    end = size
    num_steps = int((end - start) / step) + 1
    
    # Floor (Grey)
    for i in range(num_steps):
        y = start + i * step
        points.append([-size, y, 0]); points.append([size, y, 0])
        lines.append([len(points)-2, len(points)-1])
        x = start + i * step
        points.append([x, -size, 0]); points.append([x, size, 0])
        lines.append([len(points)-2, len(points)-1])
        
    # Rim Height (Red)
    offset = len(points)
    for i in range(num_steps):
        y = start + i * step
        points.append([-size, y, RIM_HEIGHT]); points.append([size, y, RIM_HEIGHT])
        lines.append([len(points)-2, len(points)-1])
        x = start + i * step
        points.append([x, -size, RIM_HEIGHT]); points.append([x, size, RIM_HEIGHT])
        lines.append([len(points)-2, len(points)-1])

    colors = [[0.2, 0.2, 0.2] for _ in range(offset)] + [[0.5, 0.1, 0.1] for _ in range(len(lines)-offset)]
    grid = o3d.geometry.LineSet()
    grid.points = o3d.utility.Vector3dVector(points)
    grid.lines = o3d.utility.Vector2iVector(lines)
    grid.colors = o3d.utility.Vector3dVector(colors)
    return grid

def create_robot_mesh():
    robot_box = o3d.geometry.TriangleMesh.create_box(0.25, 0.25, 0.075)
    robot_box.translate([-0.2, -0.15, 0])
    robot_box.paint_uniform_color([0.1, 0.1, 0.9]) 
    dustbin = o3d.geometry.TriangleMesh.create_cylinder(radius=0.06, height=0.12)
    dustbin.translate([-0.05, 0, 0.075])
    dustbin.paint_uniform_color([0.9, 0.2, 0.2]) 
    robot_mesh = robot_box + dustbin
    robot_mesh.compute_vertex_normals()
    return robot_mesh

def compute_robot_center(row):
    p4 = None
    if not pd.isna(row['tag4_x']) and str(row['tag4_x']) != '':
        p4 = np.array([float(row['tag4_x']), float(row['tag4_y']), float(row['tag4_z'])])
    p5 = None
    if not pd.isna(row['tag5_x']) and str(row['tag5_x']) != '':
        p5 = np.array([float(row['tag5_x']), float(row['tag5_y']), float(row['tag5_z'])])

    shift_m = 0.096
    default_y_axis = np.array([0.0, 1.0, 0.0])
    if p4 is not None and p5 is not None: return 0.5 * (p4 + p5)
    if p4 is not None: return p4 + (-shift_m) * default_y_axis
    if p5 is not None: return p5 + (shift_m) * default_y_axis
    return None

def calculate_landing_point(state_vec):
    x0, y0, z0, vx, vy, vz = state_vec
    if z0 < RIM_HEIGHT: return None
    a = -0.5 * GRAVITY
    b = vz
    c = z0 - RIM_HEIGHT
    disc = b**2 - 4*a*c
    if disc < 0: return None
    sqrt_disc = np.sqrt(disc)
    t1 = (-b + sqrt_disc) / (2*a)
    t2 = (-b - sqrt_disc) / (2*a)
    times = [t for t in [t1, t2] if t > 0]
    if not times: return None
    t_impact = min(times)
    pred_x = x0 + vx * t_impact
    pred_y = y0 + vy * t_impact
    return np.array([pred_x, pred_y, RIM_HEIGHT])

def predict_landing(state_vec, dt_step=0.02):
    x, y, z, vx, vy, vz = state_vec
    traj_points = []
    if z < RIM_HEIGHT and vz < 0: return []
    curr_x, curr_y, curr_z = x, y, z
    curr_vx, curr_vy, curr_vz = vx, vy, vz
    sim_t = 0
    while curr_z > 0 and sim_t < 2.0: 
        traj_points.append([curr_x, curr_y, curr_z])
        if curr_z <= RIM_HEIGHT and curr_vz < 0: break
        curr_x += curr_vx * dt_step
        curr_y += curr_vy * dt_step
        curr_z += curr_vz * dt_step
        curr_vz -= GRAVITY * dt_step
        sim_t += dt_step
    return traj_points

# ---------- Main App Logic ----------

class LogVisualizerWithPredictions:
    def __init__(self):
        # Data Loading
        self.data = self.find_and_load_csv()
        self.frame_idx = 0
        self.fps = 30
        self.is_paused = False
        self.ball_path_points = []
        
        # Toggles
        self.show_raw = True
        self.show_lkf = True
        self.show_ekf = True
        self.show_rls = True
        
        # Initialize Filters
        self.lkf = LinearKalmanFilter()
        self.ekf = ExtendedKalmanFilter()
        self.rls = RecursiveLeastSquares()
        self.filters_initialized = False
        self.prev_ts = None
        
        # --- Geometry Setup ---
        
        self.raw_ball = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS)
        self.raw_ball.paint_uniform_color(COLOR_RAW)
        self.raw_ball.compute_vertex_normals()

        self.lkf_ball = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS*0.9)
        self.lkf_ball.paint_uniform_color(COLOR_LKF)
        self.lkf_ball.compute_vertex_normals()

        self.ekf_ball = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS*0.9)
        self.ekf_ball.paint_uniform_color(COLOR_EKF)
        self.ekf_ball.compute_vertex_normals()

        self.rls_ball = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS*0.9)
        self.rls_ball.paint_uniform_color(COLOR_RLS)
        self.rls_ball.compute_vertex_normals()
        
        self.robot_geom = create_robot_mesh()
        
        # Trails
        self.raw_trail = o3d.geometry.PointCloud()
        self.raw_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
        self.raw_trail.paint_uniform_color(COLOR_RAW)

        self.lkf_trail = o3d.geometry.PointCloud()
        self.lkf_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
        self.lkf_trail.paint_uniform_color(COLOR_LKF)

        self.ekf_trail = o3d.geometry.PointCloud()
        self.ekf_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
        self.ekf_trail.paint_uniform_color(COLOR_EKF)

        self.rls_trail = o3d.geometry.PointCloud()
        self.rls_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
        self.rls_trail.paint_uniform_color(COLOR_RLS)

        # Landing Spheres
        self.land_lkf = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS * 0.8)
        self.land_lkf.paint_uniform_color(COLOR_LKF)
        self.land_lkf.compute_vertex_normals()
        
        self.land_ekf = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS * 0.8)
        self.land_ekf.paint_uniform_color(COLOR_EKF)
        self.land_ekf.compute_vertex_normals()
        
        self.land_rls = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS * 0.8)
        self.land_rls.paint_uniform_color(COLOR_RLS)
        self.land_rls.compute_vertex_normals()
        
        # Prediction Lines
        self.pred_lines = {
            'lkf': o3d.geometry.LineSet(),
            'ekf': o3d.geometry.LineSet(),
            'rls': o3d.geometry.LineSet()
        }
        for k in self.pred_lines:
            self.pred_lines[k].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            self.pred_lines[k].lines = o3d.utility.Vector2iVector([])

        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window(window_name="Ball Filter Playground", width=1280, height=720)

        opt = self.vis.get_render_option()
        opt.point_size = 5.0
        opt.background_color = np.array([0.05, 0.05, 0.05]) 

        geoms = [
            self.raw_ball, self.robot_geom, self.raw_trail,
            self.lkf_ball, self.lkf_trail, self.land_lkf,
            self.ekf_ball, self.ekf_trail, self.land_ekf,
            self.rls_ball, self.rls_trail, self.land_rls,
            create_custom_grid(),
            o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
        ]
        for g in geoms: self.vis.add_geometry(g)
        for k in self.pred_lines: self.vis.add_geometry(self.pred_lines[k])
        
        # Add Cameras
        for i, pos in enumerate([CAM1_POS, CAM2_POS]):
            cam = o3d.geometry.TriangleMesh.create_cone(radius=0.1, height=0.2)
            cam.paint_uniform_color([0,1,0] if i==1 else [1,0,0])
            cam.compute_vertex_normals()
            t = np.eye(4); t[:3,3] = pos
            R = cam.get_rotation_matrix_from_xyz((np.pi,0,0))
            cam.rotate(R, center=(0,0,0))
            cam.transform(t)
            self.vis.add_geometry(cam)

        ctr = self.vis.get_view_control()
        ctr.set_lookat([0, 0, 0])
        ctr.set_front([0, -1, 1])
        ctr.set_up([0, 0, 1])
        ctr.set_zoom(0.8)

        self._hide_all_models()

        self.vis.register_key_callback(ord(' '), self.toggle_pause)
        self.vis.register_key_callback(ord('+'), self.increase_fps)
        self.vis.register_key_callback(ord('='), self.increase_fps)
        self.vis.register_key_callback(ord('-'), self.decrease_fps)
        
        self.vis.register_key_callback(ord('0'), self.toggle_raw)
        self.vis.register_key_callback(ord('1'), self.toggle_lkf)
        self.vis.register_key_callback(ord('2'), self.toggle_ekf)
        self.vis.register_key_callback(ord('3'), self.toggle_rls)
        
        self.vis.register_animation_callback(self.update_frame)

    def _translate_to(self, geom, pos):
        center = np.asarray(geom.get_center())
        geom.translate(pos - center)

    def _hide_all_models(self):
        self._translate_to(self.raw_ball, DUMMY_POS)
        self._translate_to(self.lkf_ball, DUMMY_POS)
        self._translate_to(self.ekf_ball, DUMMY_POS)
        self._translate_to(self.rls_ball, DUMMY_POS)
        self._translate_to(self.land_lkf, DUMMY_POS)
        self._translate_to(self.land_ekf, DUMMY_POS)
        self._translate_to(self.land_rls, DUMMY_POS)

    def find_and_load_csv(self):
        files = glob.glob("../../data/triangulation_logs/log_1764153186.csv")
        if not files:
            print("No CSV found!")
            sys.exit()
        latest = max(files, key=os.path.getmtime)
        print(f"Loading: {latest}")
        df = pd.read_csv(latest)
        cols = ['ball_3d_x', 'ball_3d_y', 'ball_3d_z', 'timestamp']
        for c in cols:
            df[c] = pd.to_numeric(df[c], errors='coerce')
        return df

    def toggle_pause(self, vis):
        self.is_paused = not self.is_paused
        print(f"Paused: {self.is_paused}")
        return False

    def increase_fps(self, vis):
        self.fps = min(65, self.fps + 5)
        print(f"FPS: {self.fps}")
        return False

    def decrease_fps(self, vis):
        self.fps = max(5, self.fps - 5)
        print(f"FPS: {self.fps}")
        return False

    def toggle_raw(self, vis):
        self.show_raw = not self.show_raw
        if not self.show_raw:
            self._translate_to(self.raw_ball, DUMMY_POS)
            self.raw_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
        self.vis.update_geometry(self.raw_ball)
        self.vis.update_geometry(self.raw_trail)
        return False

    def toggle_lkf(self, vis):
        self.show_lkf = not self.show_lkf
        if not self.show_lkf:
            self._translate_to(self.lkf_ball, DUMMY_POS)
            self._translate_to(self.land_lkf, DUMMY_POS)
            self.lkf_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            self.pred_lines['lkf'].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            self.pred_lines['lkf'].lines = o3d.utility.Vector2iVector([])
        self.vis.update_geometry(self.lkf_ball)
        self.vis.update_geometry(self.land_lkf)
        self.vis.update_geometry(self.lkf_trail)
        self.vis.update_geometry(self.pred_lines['lkf'])
        return False

    def toggle_ekf(self, vis):
        self.show_ekf = not self.show_ekf
        if not self.show_ekf:
            self._translate_to(self.ekf_ball, DUMMY_POS)
            self._translate_to(self.land_ekf, DUMMY_POS)
            self.ekf_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            self.pred_lines['ekf'].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            self.pred_lines['ekf'].lines = o3d.utility.Vector2iVector([])
        self.vis.update_geometry(self.ekf_ball)
        self.vis.update_geometry(self.land_ekf)
        self.vis.update_geometry(self.ekf_trail)
        self.vis.update_geometry(self.pred_lines['ekf'])
        return False

    def toggle_rls(self, vis):
        self.show_rls = not self.show_rls
        if not self.show_rls:
            self._translate_to(self.rls_ball, DUMMY_POS)
            self._translate_to(self.land_rls, DUMMY_POS)
            self.rls_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            self.pred_lines['rls'].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            self.pred_lines['rls'].lines = o3d.utility.Vector2iVector([])
        self.vis.update_geometry(self.rls_ball)
        self.vis.update_geometry(self.land_rls)
        self.vis.update_geometry(self.rls_trail)
        self.vis.update_geometry(self.pred_lines['rls'])
        return False

    def update_frame(self, vis):
        if self.is_paused: return False
        time.sleep(1.0 / self.fps)

        if self.frame_idx >= len(self.data):
            print(f"Looping in {LOOP_DELAY}s...")
            time.sleep(LOOP_DELAY)
            self.frame_idx = 0
            self.ball_path_points = []
            self.prev_ts = None
            self.lkf = LinearKalmanFilter()
            self.ekf = ExtendedKalmanFilter()
            self.rls = RecursiveLeastSquares()
            self.filters_initialized = False
            
            # Reset Geoms
            self.raw_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            self.lkf_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            self.ekf_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            self.rls_trail.points = o3d.utility.Vector3dVector(np.array([DUMMY_POS]))
            for k in self.pred_lines:
                self.pred_lines[k].points = o3d.utility.Vector3dVector(DUMMY_POINT)
                self.pred_lines[k].lines = o3d.utility.Vector2iVector([])
            self._hide_all_models()
            
            geoms = [self.raw_trail, self.lkf_trail, self.ekf_trail, self.rls_trail,
                     self.raw_ball, self.lkf_ball, self.ekf_ball, self.rls_ball,
                     self.land_lkf, self.land_ekf, self.land_rls, 
                     self.pred_lines['lkf'], self.pred_lines['ekf'], self.pred_lines['rls']]
            for g in geoms: vis.update_geometry(g)
            return True

        row = self.data.iloc[self.frame_idx]
        bx, by, bz = row['ball_3d_x'], row['ball_3d_y'], row['ball_3d_z']
        ts = row['timestamp']
        
        valid_ball = (not pd.isna(bx) and not pd.isna(by) and not pd.isna(bz))
        
        if valid_ball:
            pos = np.array([float(bx), float(by), float(bz)])
            
            if not self.filters_initialized:
                self.lkf.x[:3] = pos; self.ekf.x[:3] = pos
                self.lkf.last_ts = ts; self.ekf.last_ts = ts
                self.filters_initialized = True

            if self.prev_ts is not None:
                dt = ts - self.prev_ts
                
                # --- COASTING / INTERPOLATION LOGIC ---
                # If we have a significant gap (> 40ms), fill it with predictions
                if dt > GAP_THRESHOLD:
                    # Generate points every 10ms
                    steps = int(dt / INTERPOLATION_STEP)
                    for i in range(1, steps):
                        interp_ts = self.prev_ts + i * INTERPOLATION_STEP
                        
                        # 1. Kalman Predict (State evolves, but no measurement)
                        self.lkf.predict(interp_ts)
                        self.lkf.save_state_to_path()
                        
                        self.ekf.predict(interp_ts)
                        self.ekf.save_state_to_path()
                        
                        # 2. RLS Query (Polynomial evaluation)
                        self.rls.interpolate_to_path(interp_ts)

                # --- REAL MEASUREMENT UPDATE ---
                self.lkf.predict(ts); self.lkf.update(pos)
                self.ekf.predict(ts); self.ekf.update(pos)
                self.rls.update(ts, pos)
            
            self.prev_ts = ts

            # --- Update VISUALS ---
            
            # 1. RAW
            if self.show_raw:
                self._translate_to(self.raw_ball, pos)
                self.update_trail(self.raw_trail, vis, self.ball_path_points, pos)
            else:
                self._translate_to(self.raw_ball, DUMMY_POS)

            # 2. LKF
            if self.show_lkf:
                self._translate_to(self.lkf_ball, self.lkf.x[:3])
                self.update_trail(self.lkf_trail, vis, self.lkf.path, self.lkf.x[:3])
                pred = calculate_landing_point(self.lkf.x)
                self._update_pred_geom(self.land_lkf, pred, vis)
                self.update_pred_line('lkf', predict_landing(self.lkf.x), COLOR_LKF, vis)
            else:
                self._translate_to(self.lkf_ball, DUMMY_POS)

            # 3. EKF
            if self.show_ekf:
                self._translate_to(self.ekf_ball, self.ekf.x[:3])
                self.update_trail(self.ekf_trail, vis, self.ekf.path, self.ekf.x[:3])
                pred = calculate_landing_point(self.ekf.x)
                self._update_pred_geom(self.land_ekf, pred, vis)
                self.update_pred_line('ekf', predict_landing(self.ekf.x), COLOR_EKF, vis)
            else:
                self._translate_to(self.ekf_ball, DUMMY_POS)

            # 4. RLS
            if self.show_rls:
                self._translate_to(self.rls_ball, self.rls.current_state[:3])
                self.update_trail(self.rls_trail, vis, self.rls.path, self.rls.current_state[:3])
                pred = calculate_landing_point(self.rls.current_state)
                self._update_pred_geom(self.land_rls, pred, vis)
                self.update_pred_line('rls', predict_landing(self.rls.current_state), COLOR_RLS, vis)
            else:
                self._translate_to(self.rls_ball, DUMMY_POS)

            geoms = [self.raw_ball, self.lkf_ball, self.ekf_ball, self.rls_ball]
            for g in geoms: vis.update_geometry(g)

        else:
            self._translate_to(self.raw_ball, DUMMY_POS)
            vis.update_geometry(self.raw_ball)

        # Update Robot
        rpos = compute_robot_center(row)
        if rpos is not None:
            center = self.robot_geom.get_center()
            self.robot_geom.translate(rpos - center)
            vis.update_geometry(self.robot_geom)
        
        self.frame_idx += 1
        return True

    def _update_pred_geom(self, geom, pos, vis):
        if pos is not None: self._translate_to(geom, pos)
        else: self._translate_to(geom, DUMMY_POS)
        vis.update_geometry(geom)

    def update_trail(self, trail_geom, vis, path_list, new_pt):
        # Only update trail if significant move or first point
        if len(path_list) > 0:
            dist = np.linalg.norm(new_pt - path_list[-1])
            if dist < MIN_MOVE_DIST: return
        
        points = np.array(path_list)
        if len(points) > 0:
            trail_geom.points = o3d.utility.Vector3dVector(points)
            vis.update_geometry(trail_geom)
    
    def update_pred_line(self, name, points, color, vis):
        if len(points) < 2:
            self.pred_lines[name].points = o3d.utility.Vector3dVector(DUMMY_POINT)
            self.pred_lines[name].lines = o3d.utility.Vector2iVector([])
        else:
            lines = [[i, i+1] for i in range(len(points)-1)]
            self.pred_lines[name].points = o3d.utility.Vector3dVector(np.array(points))
            self.pred_lines[name].lines = o3d.utility.Vector2iVector(lines)
            self.pred_lines[name].colors = o3d.utility.Vector3dVector([color for _ in lines])
        vis.update_geometry(self.pred_lines[name])

    def run(self):
        print("Controls: Space=Pause, 0=Raw, 1=LKF, 2=EKF, 3=RLS")
        self.vis.run()
        self.vis.destroy_window()

if __name__ == "__main__":
    app = LogVisualizerWithPredictions()
    app.run()

Loading: ../../data/triangulation_logs/log_1764153186.csv
Controls: Space=Pause, 0=Raw, 1=LKF, 2=EKF, 3=RLS
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding fa